# Tutorial 4: Create datalake and Athena tables

In [ ]:
import boto3 as boto3

# Connect to Athena

In [ ]:
bucket_name = "s3://quilt-example-bucket/ccle"

In [1]:
# Connect to Athena
# conn = connect(aws_access_key_id=aws_access_key_id,
#                aws_secret_access_key=aws_secret_access_key,
#                region_name=region_name,
#                s3_staging_dir= bucket_name)

# Generate symlink datalake

In [ ]:
s3 = boto3.resource('s3')
bucket_name = "quilt-example-bucket"
prefix = "ccle/datalake"

In [ ]:
# define the quant.sf files we want to use
parse_keys = """
WITH extracted_data AS (
    SELECT
        timestamp,
        logical_key,
        regexp_extract(logical_key, '^[^/]+/([^/]+)/.*', 1) AS sample_id,
        physical_key,
        regexp_extract(physical_key, '^(.*?)\?versionId=', 1) AS object_path
    FROM
        "userathenadatabase-2htmlbiqyvry"."quilt-example-bucket_objects-view"
    WHERE
        logical_key LIKE '%quant.sf'
)
SELECT
    logical_key,
    sample_id,
    physical_key,
    object_path
FROM
    extracted_data
WHERE
    sample_id LIKE 'SRR%';
    AND timestamp = 'latest';
"""

cursor = conn.cursor()
cursor.execute(parse_keys)

In [ ]:
# generate a symlinks file for each file of interest
for row in cursor:
        sample_id = row[1]
        run_id = row[3].split("/")[4]
        object_path = row[3]
        s3.Bucket(bucket_name).put_object(Key=f"{prefix}/run_id={run_id}/sample_id={sample_id}/symlink.txt", Body=object_path)